# Poverty Analysis in NL - workshop
<a target="_blank" href="https://colab.research.google.com/github/Kickstartai-product/kai-workshops/blob/main/workshops/26032024-news-analysis/news-analysis.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Setting up

### Load data

In [ ]:
%%capture
!wget -O "nos_data.csv.zip" "https://www.dropbox.com/scl/fi/5b0kkjih4xv9rnsfzbw63/nos_data_final_pruned.csv.zip?rlkey=ytgbvwj3238dnjd33wg7yu50y&dl=1"
!wget -O "ipsos_polls.csv" "https://www.dropbox.com/scl/fi/gzsr1ocq63xez3n79x048/ipsos_polls.csv?rlkey=0hf2xqrdt6c8spw7czwi89mt9&dl=1"
!wget -O "full_model_en.zip" "https://www.dropbox.com/scl/fi/1z4rhdupham561bar3ngx/full_model_en.zip?rlkey=k3c975ol1le94tunnevvlrzs5&dl=1"
!unzip -o nos_data.csv.zip
!unzip -o full_model_en.zip
!wget -O "cbs_consumer_trust.csv" "https://www.dropbox.com/scl/fi/tre8belg4w6ujt81cimvq/cbs_consumer_trust.csv?rlkey=y4qugewaegpq0wfffz456ga8i&dl=1"


### Install dependencies

In [ ]:
!pip3 install bertopic --no-deps
!pip3 install pandas scikit-learn hdbscan umap-learn
!pip3 install sentence-transformers --no-deps
!pip3 install geopandas
!pip3 install geoplot
!pip3 install wordcloud
!pip3 install plotly

### Imports and helper functions

In [ ]:
# import libraries
import pandas as pd
import bertopic as bt
from bertopic.representation import TextGeneration
from transformers import pipeline
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd
import geoplot as gplt
import plotly.io as pio
from wordcloud import WordCloud
import re

import random
from umap import UMAP
from hdbscan import HDBSCAN

import scipy.sparse as sp

from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer

# set seed for reproducibility
SEED = 42
random.seed(SEED)
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0,
                  metric='cosine', random_state=SEED)
hdbscan_model = HDBSCAN(min_cluster_size=150, metric='euclidean',
                        cluster_selection_method='eom', prediction_data=True)

# TF-IDF Transformer helper class
class CTFIDFVectorizer(TfidfTransformer):
    def __init__(self, *args, **kwargs):
        super(CTFIDFVectorizer, self).__init__(*args, **kwargs)

    def fit(self, X: sp.csr_matrix, n_samples: int):
        """Learn the idf vector (global term weights) """
        _, n_features = X.shape
        df = np.squeeze(np.asarray(X.sum(axis=0)))
        idf = np.log(n_samples / df)
        self._idf_diag = sp.diags(idf, offsets=0,
                                  shape=(n_features, n_features),
                                  format='csr',
                                  dtype=np.float64)
        return self

    def transform(self, X: sp.csr_matrix) -> sp.csr_matrix:
        """Transform a count-based matrix to c-TF-IDF """
        X = X * self._idf_diag
        X = normalize(X, axis=1, norm='l1', copy=False)
        return X

# normalize helper function
def normalize_series(series):
    return (series - series.mean()) / series.std()


## Exercise 0: Loading the data and doing some exploration

### Loading in the dataset

This dataset has over a 150,000 news articles (rows), the followings columns are present:

* `datetime`: time of posting
* `title`: title of article
* `category`: category of the news
* `url`: URL of the article
* `embedding`: OpenAI LLM embeddings of the article (not used in the notebook, but can be used later)
* `location`, `lat`, `long`, `province`: location estimates of each article, obtained by Named Entity Recongition + Google Maps API
* `translated_content`: content of the article, translated to English using Google Translate
* `secret_cluster`: categorization of articles about poverty

In [ ]:
df = pd.read_csv('nos_data_final_pruned.csv', parse_dates=['datetime'])

# cast embeddings to numpy array
df['embedding'] = df['embedding'].apply(
    lambda x: np.fromstring(x[1:-1], sep=',', dtype=np.float16))
# sort by datetime
df = df.sort_values(by='datetime')

In [ ]:
# show first 5 rows of data frame to see what's in the data
df.head()

### Creating visualizations

Let's create some basic visualizations about 3 aspects of the data:

* The distribution of categories in the data
* The distribution of articles over time
* The spatial distribution of articles

Are there results in line with your expectation? Why/Why not?

In [ ]:
fig, ax = plt.subplots(ncols=3, figsize=(20, 5))

# Create pie plot of the categories to see the division of different categories in the data
df['category'].value_counts().iloc[:5].plot(
    kind='pie', title='Top 5 categories', xlabel='Category', ylabel='Count', ax=ax[0])

# bar plot to show the number of articles per year
df['datetime'].dt.year.value_counts().sort_index().plot(kind='bar', title='Articles per year', xlabel='Category', ylabel='Count', ax=ax[1]);
# rotate x-axis labels for readability
plt.xticks(rotation=45)

plt.tight_layout()
# Geoplot to gain insights about locations that are more often in the news, we take a
# map of the netherlands from stack.stanford.edu and plot the locations of the articles
gdf = gpd.read_file('https://stacks.stanford.edu/file/druid:st293bj4601/data.zip')
gdf = gdf[gdf['TYPE_1'] == 'Provincie']
# drop all rows where ENGTYPE_1 is not 'Province'
gdf.plot(ax=ax[2])
xlims = ax[2].get_xlim()
ylims = ax[2].get_ylim()
ax[2].scatter(df['lng'], df['lat'], color='tab:orange', alpha=0.1, s=1)
ax[2].set_xlim(*xlims)
ax[2].set_ylim(*ylims)
ax[2].set_title('Article Locations');

## Exercise 1: TF-IDF exploration

We will be applying TF-IDF on a subset of our dataset, namely, on articles directly related to poverty. The goal is to find topics (words) in the articles about poverty that are special (very important) in that certain year. Those words are commonly that year, but less common in the others.

We follow the following plan:

1. Create a slice of the data that contains only poverty articles (we have classified this data already for you). We save this slice in a temporary data frame called `poverty_df`.

2. We then divide our dataset into a separate cluster, each cluster representing articles from one calendar year. The first cluster, 2010, will be cluster 0, 2011 will be cluster 1, and so forth.

3. We then fit the data on a `CountVectorizer`, a class from `sklearn`. This class performs Bag of Words, e.g., it turns our text into vectors that contain the frequency of all unique words.

4. We then perform TF-IDF on each cluster, compared to all articles about poverty.

5. We visualize the results in a word cloud for the year 2022.

It can be initialized as follows:

`count_vectorizer = CountVectorizer(stop_words="english")`

Where stop_words is set to English, so that the bag of words method skips stop words used in the English language.



In [ ]:
# create the poverty dataframe, ignoring NaN values in the secret_cluster column
poverty_df = df.dropna(subset=['secret_cluster']).copy()
# select all rows where the secret_cluster is 1, where 1 indicates articles related to poverty
poverty_df = poverty_df[poverty_df['secret_cluster'] == 1]
# get the year for each article, save it in a new column called 'year'
poverty_df['year'] = poverty_df['datetime'].dt.year
# Group by year
poverty_df = poverty_df[['translated_content', 'year']]
# show the dataframe
poverty_df.head()

Now, we group all documents of each year together. This makes each year one big 'article', such that we can easily count the word occurance for each year.

The output you get should look like this:

|    |   year | translated_content           |
|---:|-------:|:-----------------------------|
|  0 |   2010 | Prime Minister Balkenende... |
|  1 |   2011 | By editor Sander Warmerda... |
|  2 |   2012 | One in five Greeks lives ... |
|  3 |   2013 | Hundreds of church buildi... |
| ... | ... | ... |

In [ ]:
# group by year and concatenate all articles in that year
articles_per_year = poverty_df.groupby(['year'], as_index=False).agg({'translated_content': ' '.join})
# show the dataframe (first 5 rows)
articles_per_year.head()

Now, we perform Bag of Words using sklearns CountVectorizer. We ignore English stop words, in the end, we obtain:

* `count`: An array with counts for each unique word and all the articles per year. The array has shape `(N_years, N_unique_words)`
* `word`: An array with all unique words. This array has a shape `(N_unique_words)`, there are over 17,000 unique words found!

In the end, we print an examle of 10 identified words:

`array(['judges', 'judgment', 'judgments', 'judicial', 'judiciary',
       'judikje', 'judith', 'juffermans', 'juice', 'juliana'],
      dtype=object)`

Can you spot a weakness of Bag of Words based on this output?



In [ ]:
# create a bag of words class using CountVectorizer
count_vectorizer = CountVectorizer(stop_words="english")
# get the counts of each word in the corpus, for each class
count = count_vectorizer.fit_transform(articles_per_year.translated_content)
# get the unique words in the corpus
words = count_vectorizer.get_feature_names_out()

# example of the first 10 words
words[len(words)//2: len(words)//2 + 10]

Now we perform TF-IDF, this class calculates the TF-IDF values for each article cluster, for each term, as such, it has the same shape as `count`.

This results in an array with a score for every year and every unique word.

In [ ]:
# Create a TF-IDF class, using the helper class defined in the beginning of this notebook
ctfidf = CTFIDFVectorizer().fit_transform(count, n_samples=len(poverty_df)).toarray()

ctfidf

We select a year using `selected_year`. You can choose any year in the range 2010-2023

In [ ]:
# Select a year. Note you can change the year as long as its in the range 2010-2023
selected_year =  2022

Show the top 25 terms and their TF-IDF values. Do you think it makes sense?

In [ ]:
# get values for the selected year, the index starts at 2010
ctfidf_values = ctfidf[selected_year - 2010]
# get the indices of the top 25 words
top_25_indices = ctfidf_values.argsort()[-25:][::-1]
# get the 25 words with the highest TF-IDF values
top_25_words = {words[index]: ctfidf_values[index] for index in top_25_indices}
# get the tfidf values for the top 25 words
top_25_values = [ctfidf_values[index] for index in top_25_indices]
# create a dataframe with the top 25 words and their tfidf values
top_25_df = pd.DataFrame({'word': list(top_25_words.keys()), 'tfidf': list(top_25_words.values())})

top_25_df

Now, let's make a word cloud of the terms!

In [ ]:
# Convert the list of keywords into a single string with spaces
text = " ".join(top_25_df['word'])

# Create a WordCloud object
wordcloud = WordCloud(width = 800, height = 800,
                background_color ='white',
                min_font_size = 10).generate(text)

# Display the generated image:
plt.figure(figsize = (8, 8), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)

plt.show()

## Exercise 2: BERTopic

In this exercise we will visualize the embedding space used by BERTopic. We created the embeddings already.

1. We will use BERTopic to visualize the embeddings in a UMAP, clustering similar topics. Note, the code is commented out, because it takes a long time to fit the model. However, you can run this code later.
2. Next we will look at the topics identified and what is in their representation.
3. You can visualize the embedding space. As this takes a long time we will provide the HTML, so we can check it out. Later you can run this part of the code as well.  


The results of the visualisation can be seen by clicking on this [link](https://www.dropbox.com/scl/fi/v3pvl10q6axzwyyv4jnt1/visualize_documents_top25.html?rlkey=kf9o4sgrhlogsz2caw7ki8fft&dl=1)

This is how you would fit the (clustering) model, depending on your CPU/GPU setup.
For now, we'll skip this part and load the fitted model in.

In [ ]:
""" Fit the BERTopic model """
# documents = df['translated_content'].tolist()
# embeddings = np.array(df['embedding'].tolist())

# vectorizer_model = CountVectorizer(stop_words="english")
# bertopic_model = bt.BERTopic(umap_model=umap_model, hdbscan_model=hdbscan_model, verbose=True, vectorizer_model=vectorizer_model)


Look at the topics and their representations.

Note: `-1` is a remainder topic, all articles that couldn't be fitted are stored here. For this reason we will ignore `Topic` `-1`

In [ ]:
bertopic_model = bt.BERTopic.load("full_model_en")
df['topic'] = bertopic_model.topics_
bertopic_model.get_topic_info().head(25).iloc[:, :-1]

In [ ]:
bertopic_model.get_topic_info().head(25).iloc[:, :-1]["Name"]

This is how you could generate a latent space visualization, again, we'll skip this for now as it is quite CPU intensive, and provide you the [html](https://www.dropbox.com/scl/fi/v3pvl10q6axzwyyv4jnt1/visualize_documents_top25.html?rlkey=kf9o4sgrhlogsz2caw7ki8fft&dl=1) file directly.
If you have already downloaded the file, no need to click on this link again.

In [ ]:
"""Reduce dimensionality of embeddings, this step is optional but much faster to perform iteratively:"""
# reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine', random_state=SEED).fit_transform(embeddings)
# fig = bertopic_model.visualize_documents(documents, reduced_embeddings=reduced_embeddings, topics=list(range(1, 26)))
# pio.write_html(fig, file='visualize_documents.html', auto_open=False)


## Exercise 3: Spatial distribution of topics
In this exercise we investigate how news topics are distributed geographically.
We do this by:
1. Selecting a specific topic. You can select your own by running `bertoptic_model.get_topic_info()` or choose some of the topics we suggest below.
2. Create a plot where news articles are plotted on their approximate locations.


In [ ]:
# Take a selected topic, to see all topics, run bertopic_model.get_topic_info()
# Suggestions:
selected_topic = 1

In [ ]:
# take a selected topic, to see all topics, run bertopic_model.get_topic_info()
selected_topic = 1
df_topic = df[df['topic'] == selected_topic].dropna(subset=['lng', 'lat'])

# plot the dutch geodata
# Load shapefiles of NL
gdf = gpd.read_file('https://stacks.stanford.edu/file/druid:st293bj4601/data.zip')

# take a selected topic, to see all topics, run bertopic_model.get_topic_info()
selected_topic = 11
df_topic = df[df['topic'] == selected_topic].dropna(subset=['lng', 'lat'])

# make geo dataframe, has cols lat and lng
gdf_temp = gpd.GeoDataFrame(df_topic, crs="EPSG:4326", geometry=gpd.points_from_xy(df_topic.lng, df_topic.lat, crs="EPSG:4326"))

# limit to articles with geo location in NL
points_within_gdf = gpd.sjoin(gdf_temp, gdf, how='inner', op='within')

# plot
ax = gplt.polyplot(gdf, projection=gplt.crs.AlbersEqualArea(), zorder=1)
gplt.kdeplot(
      points_within_gdf, cmap='Reds', shade=True, ax=ax,
      thresh=0.1,
      alpha=.9
    )

Why do you think a lot of these plots look similar? Do you have an idea of how to fix this?

## Exercise 4: Analysis using topics developments

In this exercise we will 1) explore how the frequencies of news mentions within a topic change over time, and 2) see if we can find correlections with the perception of economics.

To do this, we take the following steps:


1.   Get the topic counts for every month in the years 2010 to 2023. The topics are the topics found using BERTopic clustering (UMAP).
2.   Normalize the counts
3. Select a topic and plot the counts for that topic over time (months in the years 2010 to 20230).
4. Use economic perception data from the CBS and select one indicator to plot it over time.
5. Combine economic perception data and a selected topic from our news data and see if there are any correlations vissible.




First we create a dataframe where for every month in the years 2010 to 2023 the topic mentions are counted.
In other words, we count how many times a topic e.g., `4_students_education_school_schools` is mentioned in a specific month.

In [ ]:
# get calendar months
df['month'] = df['datetime'].dt.to_period('M').dt.to_timestamp()
# group by month and topic and count the number of articles
monthly_topic_counts = df.groupby(['month', 'topic']).size().unstack().fillna(0).iloc[:, 1:]
# set the BERTopic topic names as columns
monthly_topic_counts.columns = bertopic_model.get_topic_info()['Name'].iloc[1:]

# visualize a part of the newly created data frame to see what's in it.
monthly_topic_counts.iloc[:10, :5]

Normalize the topic counts.

In [ ]:
# normalize the counts by dividing by the total number of articles in that month
normalized_monthly_topic_counts = monthly_topic_counts.div(monthly_topic_counts.sum(axis=1), axis=0)

# visualize part of the data frame, to see the normalized counts
normalized_monthly_topic_counts.iloc[:10, :5]

Choose a topic to plot.
We have made a selection of interesting topics you can choose from:

But, you can also choose any topic yourself as long as it was found by the BERTopic clustering we did before `bertopic_model.get_topic_info()['Name']`.


In [ ]:
# Set your own selected topic to visualize
selected_topic = '0_patients_care_healthcare_hospital'

Plot `selected_topic` over time.

In [ ]:
normalized_monthly_topic_counts[selected_topic].plot(xlabel='Month', ylabel='Count', title='Monthly article count for topic: ' + selected_topic);

Next we will look at the economic perception data from the CBS. We will first load the data into a data frame. Then we will set the index of the data to months and look at the variable `Koopbereidheid` by plotting it.

In [ ]:
# load CBS data in data frame
df_cbs = pd.read_csv('cbs_consumer_trust.csv', sep=';')
# set index to month
df_cbs['month'] = pd.to_datetime(df_cbs['Perioden'], format='%YMM%m')
df_cbs.set_index('month', inplace=True)
# plot the Koopbereidheid variable
df_cbs['Koopbereidheid'].plot()

Now we will combine the plots from our news data with the CBS plot, to see if we can (visually) identify any correlations.

As an example we have used the following columns from the data set:
- From the news data we use the topic `16_housing_mortgage_homes_rental`
- From the CBS data we use the feature `EconomischeSituatieKomende12Maanden`

However, you can use any topics and features you want to explore if you can find any other interesting correlations.

In [ ]:
# Select the topic(s) from the news data you want to visualize
selected_topics = ['16_housing_mortgage_homes_rental']

# Select a CBS feature you want to plot
cbs_feature = 'EconomischeSituatieKomende12Maanden'


In [ ]:
# create a combined data frame of the selected news topic(s) and the cbs_feature
plot_df = normalized_monthly_topic_counts[selected_topics].join(-df_cbs[cbs_feature])
# apply normalization to the columns
plot_df = plot_df.apply(normalize_series)
# apply 3 month rolling average
plot_df = plot_df.rolling(6).mean()

# plot the data frame
plot_df.plot(xlabel='Month', ylabel='Normalized count', title='Monthly article count for topic: ' + selected_topic + ' and consumer trust');

In [ ]:
# TODO: add some possible interesting things participants can do


## Survey

Please let us know what you think about this workshop by filling in our [survey](https://workshop-march26.kickstartai-events.org/page/1498843).

Thank you!

### Backup

Loading data


In [ ]:
%%capture
# nos_data.csv.zip
!gdown 1NZf16cNgIqKhQvVzqW65Nivds_tWPxuE

# ipsos_polls.csv
!gdown 1qwBMbZvo9R6QDKhiJIyOa8f28h5Hfydu

# full_model_en.zip
!gdown 1Pze19r-k0wvzN6ZemNwCWeYz3A1nRPgG

# cbs_consumer_trust.csv
!gdown 1oqMIHMNNLVjyrOSDwbOXAZw4sVCkgIqZ

!unzip -o nos_data.csv.zip
!unzip -o full_model_en.zip